In [16]:
import pandas as pd
import numpy as np

data = "D:\\Matana\\Semester 4\\Metode Numerik\\BostonHousing.csv"
df = pd.read_csv(data)
df

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0


## setiap variabel memiliki arti, dimana :
- crim : rate kriminal per kapita by town
- zn : Proporsi kavling hunian yang luas (lebih dari 25.000 kaki persegi)
- indus: Proporsi luas lahan bisnis non-ritel per kota.
- Chas: Variabel biner yang menunjukkan apakah properti tersebut dekat dengan Sungai Charles (1 untuk ya, 0 untuk tidak).
- nox: konsentrasi nitrogen oksida di udara
- rm: jumlah rata-rata kamar per hunian
- age: Proporsi unit-unit yang dihuni pemilik lama yang dibangun sebelum tahun 1940.
- dis: Jarak tertimbang ke pusat-pusat pekerjaan di Boston.
- rad: Indeks aksesibilitas ke jalan raya radial.
- tax: Tarif pajak properti per $10.000.

In [17]:
na_locations = df.isna()
for index, row in na_locations.iterrows():
    for col in df.columns:
        if row[col]:
            print(f"NA ditemukan di baris {index}, kolom '{col}'")

NA ditemukan di baris 10, kolom 'rm'
NA ditemukan di baris 35, kolom 'rm'
NA ditemukan di baris 63, kolom 'rm'
NA ditemukan di baris 96, kolom 'rm'
NA ditemukan di baris 135, kolom 'rm'


In [2]:
df['rm'].fillna(df['rm'].median(), inplace=True)
X = df[['rm', 'ptratio', 'lstat']].values
y = df['medv'].values.reshape(-1, 1)

C:\Users\Upank\AppData\Local\Temp\ipykernel_7668\2441513519.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['rm'].fillna(df['rm'].median(), inplace=True)


## Hal di atas dilakukan untuk memilih fitur dan target

In [7]:
X = np.hstack((np.ones((X.shape[0], 1)), X))

In [21]:
X_T = X.T
print("X Transpose:\n",(X_T))

X Transpose:
 [[ 1.     1.     1.    ...  1.     1.     1.   ]
 [ 6.575  6.421  7.185 ...  6.976  6.794  6.03 ]
 [15.3   17.8   17.8   ... 21.    21.    21.   ]
 [ 4.98   9.14   4.03  ...  5.64   6.48   7.88 ]]


In [24]:
XT_X = np.round(X.T @ X, 2)

print("X^T X:\n", XT_X)

X^T X:
 [[   506.     3179.5    9338.5    6402.45]
 [  3179.5   20227.59  58405.89  38674.09]
 [  9338.5   58405.89 174713.93 121080.91]
 [  6402.45  38674.09 121080.91 106762.96]]


In [10]:
XT_y = X.T @ y

print("X^T y:\n", XT_y)

X^T y:
 [[ 11401.6   ]
 [ 73911.5544]
 [205316.73  ]
 [119799.159 ]]


In [25]:
# Implementasi metode Jacobi dan Gauss-Seidel
def jacobi(A, b, tol=1e-6, max_iter=1000):
    x = np.zeros_like(b, dtype=np.float64)
    D = np.diag(A)
    R = A - np.diagflat(D)
    
    for i in range(max_iter):
        x_new = (b - np.dot(R, x)) / D
        if np.linalg.norm(x_new - x, ord=np.inf) < tol:
            return x_new, i
        x = x_new
    return x, max_iter


def gauss_seidel(A, b, tol=1e-6, max_iter=1000):
    x = np.zeros_like(b, dtype=np.float64)
    for k in range(max_iter):
        x_new = np.copy(x)
        for i in range(A.shape[0]):
            sum1 = np.dot(A[i, :i], x_new[:i])
            sum2 = np.dot(A[i, i+1:], x[i+1:])
            x_new[i] = (b[i] - sum1 - sum2) / A[i, i]
        if np.linalg.norm(x_new - x, ord=np.inf) < tol:
            return x_new, k
        x = x_new
    return x, max_iter

# Jalankan metode Jacobi dan Gauss-Seidel
sol_jacobi, iter_jacobi = jacobi(XT_X, XT_y)
sol_gauss, iter_gauss = gauss_seidel(XT_X, XT_y)

# Tampilkan hasil
print(f"Metode Jacobi: {iter_jacobi} iterasi")
print(f"Metode Gauss-Seidel: {iter_gauss} iterasi")
print("Solusi Jacobi:\n", np.round(sol_jacobi, 2))
print("Solusi Gauss-Seidel:\n", np.round(sol_gauss, 2))

Metode Jacobi: 1000 iterasi
Metode Gauss-Seidel: 1000 iterasi
Solusi Jacobi:
 [[             nan              nan  0.00000000e+000 -1.83010129e+155]
 [             nan              nan  3.00000000e-002 -1.00422266e+156]
 [             nan              nan  1.34000000e+000 -1.60680176e+156]
 [             nan              nan -2.50000000e-001 -1.53255893e+156]]
Solusi Gauss-Seidel:
 [[18.76]
 [ 4.5 ]
 [-0.94]
 [-0.57]]


In [27]:
np.all(np.abs(np.diag(XT_X)) > np.sum(np.abs(XT_X), axis=1) - np.abs(np.diag(XT_X)))

np.False_